In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
sf_strt = pd.read_csv('./street_data/processed_strt_data.csv')

In [3]:
sf_strt.head(2)

,Unnamed: 0,classcode,oneway,streetname,cnn,from_cnn,to_cnn,from_cnn_coords,to_cnn_coords,length
0,0,5,B,GRANT AVE,6408000,25376000,25391000,"[-122.40789570242603, 37.802203159671244]","[-122.40789866598965, 37.80221802045265]",1.671781
1,1,5,B,HYDE ST,7141000,30028000,25543000,"[-122.41910554731129, 37.79948227457389]","[-122.41910945900483, 37.799501833031165]",2.200408


In [4]:
sf_strt.drop(labels='Unnamed: 0', axis=1, inplace=True)

In [5]:
sf_strt.head(2)

,classcode,oneway,streetname,cnn,from_cnn,to_cnn,from_cnn_coords,to_cnn_coords,length
0,5,B,GRANT AVE,6408000,25376000,25391000,"[-122.40789570242603, 37.802203159671244]","[-122.40789866598965, 37.80221802045265]",1.671781
1,5,B,HYDE ST,7141000,30028000,25543000,"[-122.41910554731129, 37.79948227457389]","[-122.41910945900483, 37.799501833031165]",2.200408


In [6]:
sf_strt[sf_strt['from_cnn']==20245000]

,classcode,oneway,streetname,cnn,from_cnn,to_cnn,from_cnn_coords,to_cnn_coords,length
3159,3,F,03RD ST,200202,20245000,20244000,"[-122.38750874616855, 37.74406207957941]","[-122.38762945352309, 37.74352792755228]",60.297955
3196,5,F,PHELPS ST,10394000,20245000,20246000,"[-122.38750874616855, 37.74406207957941]","[-122.38790040732846, 37.743610176527405]",60.879478


In [7]:
df_b = sf_strt[['oneway','cnn','from_cnn','to_cnn']][sf_strt['oneway']=='B']
df_b.reset_index(inplace=True)
df_t = sf_strt[['oneway','cnn','from_cnn','to_cnn']][sf_strt['oneway']=='T']
df_t.reset_index(inplace=True)
df_f = sf_strt[['oneway','cnn','from_cnn','to_cnn']][sf_strt['oneway']=='F']
df_f.reset_index(inplace=True)

In [8]:
df_b.head(2)

,index,oneway,cnn,from_cnn,to_cnn
0,0,B,6408000,25376000,25391000
1,1,B,7141000,30028000,25543000


In [9]:
def collect(x):
    return list(x)

In [10]:
df_B1 = pd.DataFrame(df_b.groupby('from_cnn').agg({'to_cnn':collect}))
#df_T1 = df_t.groupby('from_cnn').agg({'to_cnn':collect})
df_F = df_f.groupby('from_cnn').agg({'to_cnn':collect})

In [11]:
df_B2 = df_b.groupby('to_cnn').agg({'from_cnn':collect})
df_T = df_t.groupby('to_cnn').agg({'from_cnn':collect})
#df_F2 = df_f.groupby('to_cnn').agg({'from_cnn':collect})

In [12]:
df_B1.reset_index(inplace=True)
df_B1.rename(columns={'from_cnn':'node'},inplace=True)
df_T.reset_index(inplace=True)
df_F.reset_index(inplace=True)

In [13]:
df_B2.reset_index(inplace=True)
df_B2.rename(columns={'to_cnn':'node'},inplace=True)

#df_T2.reset_index(inplace=True)
#df_F2.reset_index(inplace=True)

In [14]:
df_B1.head(2)

,node,to_cnn
0,20010000,[20435000]
1,20011000,"[20101000, 20097000]"


In [15]:
df_B2.head(2)

,node,from_cnn
0,20010000,"[51652000, 28144000]"
1,20013000,"[20101000, 51653000]"


In [16]:
df_B2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7978 entries, 0 to 7977
Data columns (total 2 columns):
node        7978 non-null int64
from_cnn    7978 non-null object
dtypes: int64(1), object(1)
memory usage: 124.7+ KB


In [17]:
def combine(row, col1='to_cnn', col2='from_cnn'):      
    if row[col1] is np.nan and row[col2] is np.nan:
        return [[]]
    
    if row[col1] is np.nan:
        return [row[col2]]
    
    if row[col2] is np.nan:
        return [row[col1]]
    
    #print type(row['node_neighbors_y'])
    #print type(row)
  
    return [row[col1]+row[col2]]
    
    

In [18]:
df_B1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7855 entries, 0 to 7854
Data columns (total 2 columns):
node      7855 non-null int64
to_cnn    7855 non-null object
dtypes: int64(1), object(1)
memory usage: 122.8+ KB


In [19]:
df_B3 = df_B1.merge(df_B2,how='outer',left_on='node',right_on='node')


In [20]:
df_B3.head(2)

,node,to_cnn,from_cnn
0,20010000.0,[20435000],"[51652000, 28144000]"
1,20011000.0,"[20101000, 20097000]",NaN


In [21]:
df_B3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8986 entries, 0 to 8985
Data columns (total 3 columns):
node        8986 non-null float64
to_cnn      7855 non-null object
from_cnn    7978 non-null object
dtypes: float64(1), object(2)
memory usage: 280.8+ KB


In [22]:
df_B3['node_neighbors'] = df_B3.apply(combine, axis=1)

In [23]:
df_B3.drop(labels=['to_cnn','from_cnn'],axis=1,inplace=True)

In [24]:
df_B3.head(2)

,node,node_neighbors
0,20010000.0,"[[20435000, 51652000, 28144000]]"
1,20011000.0,"[[20101000, 20097000]]"


In [25]:
df_B3['node_neighbors']=df_B3.node_neighbors.map(lambda x: [item for sublist in x for item in sublist])


In [26]:
df_B3.rename(columns={'from_cnn_x':'node'}, inplace=True)
df_B3.head(2)

,node,node_neighbors
0,20010000.0,"[20435000, 51652000, 28144000]"
1,20011000.0,"[20101000, 20097000]"


In [27]:
df_B3.shape

(8986, 2)

In [28]:
df_B3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8986 entries, 0 to 8985
Data columns (total 2 columns):
node              8986 non-null float64
node_neighbors    8986 non-null object
dtypes: float64(1), object(1)
memory usage: 210.6+ KB


In [29]:
df_T.rename(columns={'to_cnn':'node'}, inplace=True)
df_T.head()

,node,from_cnn
0,20181000,[20177000]
1,20196000,[33204000]
2,20197000,[20196000]
3,20198000,"[20200000, 34010000]"
4,20200000,[33634000]


In [30]:
df_F.rename(columns={'from_cnn':'node'}, inplace=True)
df_F.head(2)

,node,to_cnn
0,20177000,[20181000]
1,20181000,[20203000]


In [31]:
df_F[df_F['node']==20245000]

,node,to_cnn
20,20245000,"[20244000, 20246000]"


In [32]:
df_final1 = df_B3.merge(df_T, how='outer',left_on='node',right_on='node')

In [33]:
df_final = df_final1.merge(df_F, how='outer',left_on='node',right_on='node')

In [34]:
df_final.head(2)

,node,node_neighbors,from_cnn,to_cnn
0,20010000.0,"[20435000, 51652000, 28144000]",NaN,NaN
1,20011000.0,"[20101000, 20097000]",NaN,NaN


In [35]:
df_final[df_final['node']==20245000]

,node,node_neighbors,from_cnn,to_cnn
9344,20245000.0,NaN,NaN,"[20244000, 20246000]"


In [36]:
from functools import partial

df_final['node_neighbors'] = df_final.apply(partial(combine, col1='node_neighbors', col2='from_cnn'), axis=1)

In [37]:
df_final['node_neighbors'] = df_final.apply(partial(combine, col1='node_neighbors', col2='to_cnn'), axis=1)

In [38]:
df_final['node_neighbors'] = df_final['node_neighbors'].map(lambda x: x[0][0])

In [39]:
df_final[df_final['node']==20245000]

,node,node_neighbors,from_cnn,to_cnn
9344,20245000.0,[],NaN,"[20244000, 20246000]"


In [40]:
df_final.drop(labels=['from_cnn','to_cnn'],axis=1, inplace=True)
df_final['node'] = df_final['node'].map(lambda x: int(x))

In [41]:
df_final.head()

,node,node_neighbors
0,20010000,"[20435000, 51652000, 28144000]"
1,20011000,"[20101000, 20097000]"
2,20013000,"[51652000, 20101000, 51653000]"
3,20034000,"[33121000, 35028000, 50752000]"
4,20039000,"[20044000, 33128000, 33121000]"


In [42]:
streets = zip(list(sf_strt['to_cnn']), list(sf_strt['from_cnn']), list(sf_strt['oneway']))
streets_set = set()

for t, f, o in streets:
    if o == 'B':
        streets_set.add((f, t))
        streets_set.add((t, f))
    elif o == 'T':
        streets_set.add((t, f))
    else:
        streets_set.add((f, t))

In [43]:
def pred(row):
    node = row['node']
    neighbors = row['node_neighbors']
    
    for n in neighbors:
        if (node, n) not in streets_set:
            return False
        
    return True
    

In [46]:
df_final.apply(pred, axis=1).all()

True

In [49]:
df_final.to_json("./street_data/node_neighbors.json", orient='records')